<a href="https://colab.research.google.com/github/sauvatu/HS-algorithm/blob/master/Chp4_Word2Vec_Improvement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Word2Vec Improvement1**

If extracting a specific row, specify the desired row.

In [ ]:
import numpy as np

W = np.arange(21).reshape(7, 3)
print(W)

# Get second row
# -> index=2 Word vector corresponding to
print('Printing content from index 2: ', + W[2])
#Printing index 5
print('Printing content from index 5: ', + W[5])

# Multiple rows from weight W
# Example of extracting all at once
idx = np.array([1, 0, 3, 0])
W[idx]

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
Printing content from index 2:  [6 7 8]
Printing content from index 5:  [15 16 17]


array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

## **Implementation of Embedding Layer**

In [ ]:
#Initializes the embedding layer with a single parameter W, which represents the embedding matrix.
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

#Retrieves the embedding vectors corresponding to the given indices from the embedding matrix W.
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out

#Updates dW by accumulating gradients based on the indices idx and the corresponding gradients dout.
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        np.add.at(dW, self.idx, dout)
        return None

Implementation Embedding Dot Layer

In [ ]:
#It initializes an instance of the Embedding class with the provided embedding matrix W.
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

#Retrieves the embeddings corresponding to the indices using the Embedding layer.
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
#Computes gradients for the target embeddings (target_W) by multiplying dout with h.
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

Example of Sampling based on probability distribution.

In [ ]:
# Sampling example based on probability distribution
import numpy as np

# Randomly samples one of the numbers 0 to 9
print(np.random.choice(10))
print(np.random.choice(10))

# Randomly sample only one from words
words = ['you', 'say', 'goodbye', 'i', 'hello', '.']
print(np.random.choice(words))

print(np.random.choice(words, size=5))
# Randomly sampled only (no duplicates)
print(np.random.choice(words, size=5, replace=False))

# Sampling according to probability distribution
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
print(np.random.choice(words, p=p))

4
8
i
['you' 'i' 'i' 'you' 'hello']
['you' 'goodbye' 'i' 'hello' '.']
say


In sampling, apply 0.75 squared
to values of the probability distribution.

*   In order not to throw away words that have a low probability of appearing



In [ ]:
p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)

new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


# **Negative Sampling Implementation**

In [ ]:
import sys
sys.path.append('..')
from common.np import *
from common.layers import Embedding, SigmoidWithLoss
import collections

# Negative Sampling class implementation
# chap04/negative_sampling_layer.py
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:  # == CPU
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0  #To prevent it from being pulled out
                p /= p.sum()  # Normalize again
                negative_sample[i, :] = np.random.choice(self.vocab_size,
                                                         size=self.sample_size,
                                                         replace=False, p=p)

        else:
            # GPU(cupy)When calculating, speed takes priority.
            # Negative examples may include targets.
            negative_sample = np.random.choice(self.vocab_size,
                                               size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample



In [ ]:
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[0 3]
 [1 2]
 [2 3]]


In [ ]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections
# chap04/negative_sampling_layer.py
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]
#Creates a UnigramSampler instance for negative sampling, which will be used to
#generate negative samples from the corpus.
        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)
#Computes the loss for the positive target using the dot product of h and the target embedding,
#and calculates the loss using SigmoidWithLoss.
        # Positive Yes Net Spread
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # Negative Yes Net Spread
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]  #It seems to mean that it is a target corresponding to embed_dot
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss
#Backpropagates gradients through each loss and embedding dot product layer.
    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh

4.3 **word2vec** (Improved Version) **Learning**
*   4.3.1 Implementation of CBOW Model




In [ ]:
# chap04/cbow.py
import sys
sys.path.append('..')
from common.np import *
from common.layers import Embedding
from negative_sampling_layer import NegativeSamplingLoss

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # weight initialization
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # Create Layer
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding use layers
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # Gather all weights and gradients into an array.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # Store the distributed representation of the word in an instance variable.
        self.word_vecs1 = W_in
        self.word_vecs2 = W_out

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)  # average
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

# **CBOW** Training Code

In [ ]:
# chap04/train.py
import sys
sys.path.append('..')
import numpy as np
from common import config
# Uncomment below to run on GPU (requires CuPy).
# ===============================================
config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
#from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb

# Hyperparameter settings
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# read data
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# Create models, etc.
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# Start learning
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()
pass

In [33]:
import sys
sys.path.append('..')
import pickle
from common.util import most_similar, analogy
pkl_file = './cbow_params.pkl'
with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
word_vecs = params['word_vecs']
word_to_id = params['word_to_id']
id_to_word = params['id_to_word']
# Pick the most similar word
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)


[query] you
 we: 0.6103515625
 someone: 0.59130859375
 i: 0.55419921875
 something: 0.48974609375
 anyone: 0.47314453125

[query] year
 month: 0.71875
 week: 0.65234375
 spring: 0.62744140625
 summer: 0.6259765625
 decade: 0.603515625

[query] car
 luxury: 0.497314453125
 arabia: 0.47802734375
 auto: 0.47119140625
 disk-drive: 0.450927734375
 travel: 0.4091796875

[query] toyota
 ford: 0.55078125
 instrumentation: 0.509765625
 mazda: 0.49365234375
 bethlehem: 0.47509765625
 nissan: 0.474853515625


In [34]:
# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)

--------------------------------------------------

[analogy] king:man = queen:?
 woman: 5.16015625
 veto: 4.9296875
 ounce: 4.69140625
 earthquake: 4.6328125
 successor: 4.609375

[analogy] take:took = go:?
 went: 4.55078125
 points: 4.25
 began: 4.09375
 comes: 3.98046875
 oct.: 3.90625

[analogy] car:cars = child:?
 children: 5.21875
 average: 4.7265625
 yield: 4.20703125
 cattle: 4.1875
 priced: 4.1796875

[analogy] good:better = bad:?
 more: 6.6484375
 less: 6.0625
 rather: 5.21875
 slower: 4.734375
 greater: 4.671875
